In [1]:
def quo(a, p):
    return G(real(a/p).round() + I*imag(a/p).round())

def mod(a, p):
    return a - p*quo(a, p)

def gpow(a, e, m):
    r = G(1)
    while e:
        if e & 1:
            r = mod(r*a, m)
        a = mod(a*a, m)
        e //= 2
    return r

def sqroot(y, p, all=True):
    n = p.norm() # assert prime
    d = gpow(y, (n-1)//4, p)
    r = gpow(y, (n+3)//8, p) # r² = dy

    if d == mod(1, p):
        return [r, -r] if all else r
    
    if d == mod(-1, p):
        r *= gpow(2, (n-1)//4, p)
        return [r, -r] if all else r

    raise ArithmeticError("Impossible case")

def modint(A, N):
    # A = a + ib
    # P = x + iy
    # U = u + iv

    # b = Im PU = xv + yu
    _, v, u = xgcd(N.real(), N.imag())
    A -= N*(u + I*v)*A.imag()
    return A.real() % N.norm()

In [2]:
import itertools

def small_roots(f, bounds, m=1, d=None):
	if not d:
		d = f.degree()

	R = f.base_ring()
	N = R.cardinality()
	
	f /= f.coefficients().pop(0)
	f = f.change_ring(ZZ)

	G = Sequence([], f.parent())
	for i in range(m+1):
		base = N^(m-i) * f^i
		for shifts in itertools.product(range(d), repeat=f.nvariables()):
			g = base * prod(map(power, f.variables(), shifts))
			G.append(g)

	B, monomials = G.coefficient_matrix()
	monomials = vector(monomials)

	factors = [monomial(*bounds) for monomial in monomials]
	for i, factor in enumerate(factors):
		B.rescale_col(i, factor)

	B = B.dense_matrix().LLL()

	B = B.change_ring(QQ)
	for i, factor in enumerate(factors):
		B.rescale_col(i, 1/factor)

	H = Sequence([], f.parent().change_ring(QQ))
	for h in filter(None, B*monomials):
		H.append(h)
		I = H.ideal()
		if I.dimension() == -1:
			H.pop()
		elif I.dimension() == 0:
			roots = []
			for root in I.variety(ring=ZZ):
				root = tuple(R(root[var]) for var in f.variables())
				roots.append(root)
			return roots

	return []

In [3]:
G = GaussianIntegers()
I = G.1

In [4]:
out = {"l": 256, "n": "-14071172672595565156007768487757352152764859476382183308954294492282186438059768663873855254160723159027292054036744986124692686209556946009925974804689837 - 6610317000442669421334350882238890617453420384375090003163635532360574332331395032179627075882638211898034449832265707815889692261667583698725230403922888*I", "x": "-7176323363776769500753056510533605236945282247634744933989912054917613964436015627240387012609750534697332139121227020338895149622592143017595990663716904 + 2094702396457769544631059140752727082508262094767499540057712562064057358492625212209337565533457698493244810916660015049968645149318701876542419400598957*I", "iv": "4d8642f93746942601e384559a826c41", "c": "89499cc6d2b2f8dfb8a61a45da8400654a6584850a51c80518edf653f3da9aab2271f028dab6ba2d99c4d6b28cbed26051abe9e34fbe6987fc9008f88d63c1060c05d3ffc59ce0af5ab4bd2bc7660600"}
l = out['l']
n = eval(out['n'])
x = eval(out['x'])
iv = bytes.fromhex(out['iv'])
c = bytes.fromhex(out['c'])

In [5]:
nx = ZZ(n.real())
ny = ZZ(n.imag())
nn = ZZ(n.norm())

x = isqrt(nn//2)
y = (nx + ny)//2

A = (x + y)//2
B = (x - y)//2

ha = isqrt(A)# >> 56 << 56
hb = isqrt(B)# >> 56 << 56

In [7]:
R.<x, y> = Zmod(nn)[]
j = modint(I, n)

P = (ha + x) + j*(hb + y)
roots = small_roots(P, [2^37, 2^37], m=1, d=5)
print(roots)

g = gcd(P(*roots[0]).lift(), nn)
print(1 < g < nn)
p = gcd(n, g)
q = n//p

np = p.norm()
nq = q.norm()
assert nn == np*nq

[(241694191228341590282227755842560560458777294460434313612601928368567561132179252004200690559702366784001462879224931911375634395518906577221215287838287794163260047847132433036520652572907874754816093603034853125108161461114535907275262838739263905002342257562378452056163571433557659433475251060381783089106, 39375382935)]
True


In [9]:
x = eval(out['x'])
mp2 = GF(np)(modint(x, p)).sqrt(all=True)
mq2 = GF(nq)(modint(x, q)).sqrt(all=True)

In [10]:
mps = sum([m2.sqrt(all=True) for m2 in mp2], [])
mqs = sum([m2.sqrt(all=True) for m2 in mq2], [])

In [12]:
from Crypto.Cipher import AES
from itertools import product
from hashlib import sha256

for mp, mq in product(mps, mqs):
    m = mod(crt([mp.lift(), mq.lift()], [np, nq]), n)
    k = sha256(f"({m.real()},{m.imag()})".encode()).digest()
    pt = AES.new(k, AES.MODE_CBC, iv).decrypt(c)

    if b'FCSC' in pt:
        print('m =', m)
        print('pt =', pt)
        print()

m = 51102897160060406157241056914008951118350496681822415655361639149525942612837*I + 39770522306286956994530311833833299798569783600651871902701060740929856137114
pt = b'FCSC{ebfebd88b798e1d8de96314a8c6a212c89c43fb69deae7c2298410e35ff53d37}\n\n\n\n\n\n\n\n\n\n'

